#PREPARAÇÃO DO AMBIENTE

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sweetviz as sv
!pip install sweetviz
!pip install plotly


In [ ]:
# Configurações de exibição para melhorar a visualização
pd.set_option('display.max_rows', 1000)  # Exibir 1000 linhas do total
pd.set_option('display.max_columns', None)  # Exibir todas as colunas
pd.set_option('display.expand_frame_repr', False)  # Não expandir o DataFrame em linhas longas
pd.set_option('display.float_format', '{:.2f}'.format)  # Formatação de números flutuantes

#CARREGANDO O DATASET ORIGINAL

In [ ]:
test = pd.read_csv('/content/test.csv')
train = pd.read_csv('/content/train.csv')

##Verificação gráfica dos dados originais

In [ ]:
# Relatório teste
relatorio = sv.analyze(test)

# Salvar o relatório em um arquivo HTML
relatorio.show_html('teste_original.html')

                                             |          | [  0%]   00:00 -> (? left)

Report teste_original.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
# Relatório treino
relatorio = sv.analyze(train)

# Salvar o relatório em um arquivo HTML
relatorio.show_html('train_original.html')

                                             |          | [  0%]   00:00 -> (? left)

Report train_original.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


#AJUSTANDO O DATASET

##Analisando a proporção TREINO X TESTE atual

In [ ]:
total = train.shape[0] + test.shape[0]

print(f'Proporção treino x teste: {(train.shape[0]*100/total):.2f}% X {(test.shape[0]*100/total):.2f}%')
total

Proporção treino x teste: 93.40% X 6.60%


162784

##Criando um dataset concatenado

Observamos a presença de dados ausentes nos dois datasets originais (train e test). Será mais prático tratar esses dados em um arquivo só, por isso realizaremos uma concatenação dos datasets originais.

Isso também nos possibilitará escolhermos a proporção entre treino x teste para criação de futuros datasets que treinarão o modelo.

**Concatenando os datasets originais**

In [ ]:
#CONCATENAR
import pandas as pd
from sklearn.model_selection import train_test_split

# Concatenando os datasets
dataset_concat = pd.concat([train, test], ignore_index=True)

**Informações do Dataset**

In [ ]:
# Verificando as dimensões do novo dataset
print(f"\nFormato do dataset: {dataset_concat.shape}")


Formato do dataset: (162784, 62)


In [ ]:
# Verificar os tipos de dados de cada coluna
tipos_dados = dataset_concat.dtypes
print("\nTipos de dados das colunas:")
print(tipos_dados)


Tipos de dados das colunas:
x0     float64
x1     float64
x2     float64
x3     float64
x4     float64
x5     float64
x6     float64
x7     float64
x8     float64
x9     float64
x10    float64
x11    float64
x12    float64
x13    float64
x14    float64
x15    float64
x16    float64
x17    float64
x18    float64
x19    float64
x20    float64
x21    float64
x22    float64
x23    float64
x24    float64
x25    float64
x26    float64
x27    float64
x28    float64
x29    float64
x30    float64
x31    float64
x32    float64
x33    float64
x34    float64
x35    float64
x36    float64
x37    float64
x38    float64
x39    float64
x40    float64
x41    float64
x42    float64
x43    float64
x44    float64
x45    float64
x46    float64
x47    float64
x48    float64
x49    float64
x50    float64
x51    float64
x52    float64
x53    float64
x54    float64
x55    float64
x56    float64
x57    float64
x58    float64
x59    float64
x60    float64
y        int64
dtype: object


**Obtendo um relatório prévio do dataset concatenado**

In [ ]:
# Relatório dataset concatenado
relatorio = sv.analyze(dataset_concat)

# Salvar o relatório em um arquivo HTML
relatorio.show_html('dataset_concat_original.html')

                                             |          | [  0%]   00:00 -> (? left)

Report dataset_concat_original.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


**Exportando o dataset concatenado (ainda sem qualquer modificação)**

Objetivo: Criar uma versão do dataset concatenado, sem qualquer modificação. Ele poderá ser utilizado como ponto de partida futuro para construção de diferentes DF's que treinarão os modelos

In [ ]:
dataset_concat.to_csv('dataset_concat_original.csv', index=False)

##Tratando dados nulos

**Nulos por coluna**

In [ ]:
# Exibir a quantidade de valores nulos por coluna
quantidade_nulos = dataset_concat.isnull().sum()
print("\nQuantidade de valores nulos por coluna:")
print(quantidade_nulos)


Quantidade de valores nulos por coluna:
x0        760
x1          1
x2          1
x3      84629
x4          1
x5          1
x6       6481
x7          1
x8       2287
x9       4505
x10       103
x11     84629
x12       760
x13       103
x14         1
x15      3199
x16     84629
x17         1
x18      5633
x19       760
x20       167
x21         1
x22         1
x23       167
x24         1
x25      3248
x26         1
x27       167
x28      5633
x29         1
x30         1
x31       141
x32     49389
x33     60032
x34         1
x35       760
x36      2003
x37       760
x38         1
x39     63603
x40         1
x41       103
x42       167
x43      3248
x44       760
x45     84629
x46         1
x47      6481
x48         1
x49         1
x50       760
x51     44917
x52       760
x53         1
x54     60032
x55       760
x56    114737
x57       103
x58         1
x59     44917
x60       760
y           0
dtype: int64


**Excluindo colunas com grande ausencia de dados (acima de 40%)**

Aqui poderemos mudar a faixa de corte, de acordo com os resultados obtidos nos modelos treinados.

As colunas 'x3', 'x11', 'x16', 'x33', 'x39', 'x45', 'x54', 'x56' serão excluidas.

In [ ]:
#EXCLUIR COLUNAS COM FALTANTES ACIMA DE 40%

colunas_excluir = ['x3', 'x11', 'x16', 'x33', 'x39', 'x45', 'x54', 'x56']

dataset_concat_v1 = dataset_concat.drop(colunas_excluir, axis=1)


**Verificando a presença de duplicados**


In [ ]:
#VERIFICANDO A PRESENÇA DE DUPLICADOS
print(dataset_concat_v1.duplicated().sum())

0


##Tratando o restante dos dados nulos

Aqui utilizaremos duas técnicas diferentes para tratar os dados ausentes:

1- Substituir pela média as colunas com distribuição normal e com a mediana as assimétricas.


2- Utilizar amostragem baseada na distribuição da coluna para preencher os valores nulos, tanto para distribuição normal quanto para assimétrica.


Aqui poderemos gerar datasets diferentes e verificar o desempenho do modelo treinado por cada um deles.

### Substituir pela média e mediana

In [ ]:
from scipy.stats import shapiro

# Função para preencher dados nulos de acordo com a distribuição da coluna
def preencher_nulos_por_distribuicao(df):
    # Loop sobre as colunas do dataset
    for coluna in df.columns:
        if df[coluna].isnull().sum() > 0:  # Apenas tratar colunas com dados nulos
            print(f"\nAnalisando coluna: {coluna}")

            # Verificar se a coluna é numérica
            if pd.api.types.is_numeric_dtype(df[coluna]):

                # Realizar o teste de normalidade de Shapiro-Wilk
                stat, p = shapiro(df[coluna].dropna())  # Teste com dados não nulos
                print(f'Estatística de Shapiro-Wilk: {stat}, p-valor: {p}')

                # Se p-valor > 0.05, assume normalidade (pode ajustar esse valor se necessário)
                if p > 0.05:
                    # Preencher com a média
                    mean_value = df[coluna].mean()
                    df[coluna].fillna(mean_value, inplace=True)
                    print(f'Preenchendo nulos de {coluna} com a média: {mean_value}')
                else:
                    # Preencher com a mediana
                    median_value = df[coluna].median()
                    df[coluna].fillna(median_value, inplace=True)
                    print(f'Preenchendo nulos de {coluna} com a mediana: {median_value}')
            else:
                print(f"A coluna {coluna} não é numérica e será ignorada.")

    return df

#teste
dataset_concat_v1 = preencher_nulos_por_distribuicao(dataset_concat_v1)

###Substituir por amostragem baseada na distribuição da coluna

In [ ]:
from scipy.stats import shapiro

# Função para preencher dados nulos com base na amostragem da distribuição da coluna
def preencher_nulos_por_amostragem(df):
    # Loop sobre as colunas do dataset
    for coluna in df.columns:
        if df[coluna].isnull().sum() > 0:  # Apenas tratar colunas com dados nulos
            print(f"\nAnalisando coluna: {coluna}")

            # Verificar se a coluna é numérica
            if pd.api.types.is_numeric_dtype(df[coluna]):

                # Realizar o teste de normalidade de Shapiro-Wilk
                stat, p = shapiro(df[coluna].dropna())  # Teste com dados não nulos
                print(f'Estatística de Shapiro-Wilk: {stat}, p-valor: {p}')

                # Se p-valor > 0.05, assume normalidade (pode ajustar esse valor se necessário)
                if p > 0.05:
                    # Preencher por amostragem de uma distribuição normal com a mesma média e desvio padrão
                    mean_value = df[coluna].mean()
                    std_dev = df[coluna].std()
                    # Amostrar valores de uma distribuição normal
                    n_nulos = df[coluna].isnull().sum()
                    df[coluna].fillna(pd.Series(np.random.normal(mean_value, std_dev, n_nulos)), inplace=True)
                    print(f'Preenchendo nulos de {coluna} por amostragem normal com média: {mean_value} e desvio: {std_dev}')
                else:
                    # Preencher com amostragem da distribuição existente
                    n_nulos = df[coluna].isnull().sum()
                    df[coluna].fillna(pd.Series(np.random.choice(df[coluna].dropna(), size=n_nulos)), inplace=True)
                    print(f'Preenchendo nulos de {coluna} com base na amostragem da própria coluna')
            else:
                print(f"A coluna {coluna} não é numérica e será ignorada.")

    return df

#teste
dataset_concat_v1 = preencher_nulos_por_amostragem(dataset_concat_v1)

#OUTLIERS

# PCA

##Verificando a importância das variáveis para a saída utilizando correlação

**Importando a correlação entre as variáveis e a saída**

criando uma série com as correlações e ordenando em ordem de importância.


In [ ]:
correlacao = dataset_concat.corr(method='pearson')


# Importância de 'y' com todas as variáveis, exceto 'y'(produziria 1)
correlacao_target = correlacao['y'].drop('y')

# Ordenar em ordem decrescente de importância
variaveis_importantes = correlacao_target.abs().sort_values(ascending=False)

# Exibir todas as variáveis em sequência de importância (decrescente)
print(f'Variáveis ordenadas por importância em relação à saída: \n{variaveis_importantes}')

Variáveis ordenadas por importância em relação à saída: 
x0    0.25
x19   0.20
x18   0.19
x8    0.17
x21   0.15
x4    0.13
x36   0.12
x17   0.12
x10   0.10
x41   0.10
x3    0.09
x60   0.08
x13   0.08
x43   0.07
x30   0.07
x2    0.07
x42   0.07
x14   0.07
x37   0.06
x45   0.06
x55   0.06
x53   0.05
x25   0.05
x31   0.05
x12   0.05
x28   0.05
x27   0.05
x16   0.05
x24   0.04
x9    0.03
x50   0.03
x46   0.03
x47   0.03
x58   0.03
x38   0.03
x33   0.03
x40   0.03
x5    0.03
x51   0.03
x15   0.02
x32   0.02
x49   0.02
x34   0.02
x7    0.01
x1    0.01
x23   0.01
x52   0.01
x57   0.01
x11   0.01
x54   0.01
x22   0.01
x44   0.01
x20   0.00
x6    0.00
x59   0.00
x56   0.00
x29   0.00
x39   0.00
x26   0.00
x35   0.00
x48   0.00
Name: y, dtype: float64


**Pareto de importância das variáveis**

Mostrando o quanto as variáveis são significantes para o modelo.

Podemos observar que as 32 primeiras variáveis representam 80% da importância, enquanto o restante das variáveis (29), tem menos de 20% do total de importância.

Considerando um bom número em 95%, podemos concluir que pelo menos 13 variáveis são de menor importância, representando juntas menos de 5%.

Tal observação tem o objetivo não de excluir essas variáveis, mas de justificar a possibilidade de aplicar uma redução de dimensionalidade com PCA.

In [ ]:
import plotly.graph_objects as go

# Gráfico de barras com Plotly para as variáveis
fig = go.Figure()

# Adicionar as barras
fig.add_trace(go.Bar(
    x=variaveis_importantes.index,
    y=variaveis_importantes,
    name='Importância (Correlação)',
    marker_color='blue'
))

# Adicionar a linha cumulativa
fig.add_trace(go.Scatter(
    x=variaveis_importantes.index,
    y=soma_cumulativa,
    name='Importância Cumulativa (%)',
    mode='lines+markers',
    line=dict(color='orange', dash='dot')
))

# Ajustar layout do gráfico
fig.update_layout(
    title='Gráfico de Pareto da Importância das Variáveis',
    xaxis_title='Variáveis',
    yaxis_title='Importância (Correlação)',
    yaxis2=dict(
        title='Importância Cumulativa (%)',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.1, y=1.1, orientation="h"),
    hovermode="x"
)

# Exibir o gráfico interativo
fig.show()

**PCA**

In [ ]:
#Esse trecho só será executado quando todos os dados faltantes forem tratados.

'''from sklearn.decomposition import PCA

# Supondo que dataset_concat seja seu DataFrame com os dados normalizados
X = X = dataset_concat.drop(columns=['y'])  # desconsiderando a variável y do dataset

# Passo 1: Definir e aplicar o PCA
pca = PCA(n_components=0.95)  # Mantém 95% da variância
X_pca = pca.fit_transform(X)

# Passo 2: Criar um DataFrame para os componentes principais
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
df_pca = pd.DataFrame(X_pca, columns=pca_columns)

# Passo 3: Concatenar as novas colunas de componentes principais com o dataset original
dataset_concat_pca = pd.concat([dataset_concat, df_pca], axis=1)

# Exibir as primeiras linhas do novo dataset com as componentes principais
print(dataset_concat_pca.head())
'''

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

#Redistribuir os dados antes concatenados

In [ ]:
#utilizar após o tratamento de nulos e outliers
'''
#RANDOMIZAR E REDISTRIBUIR OS DATASETS
# Embaralhando e dividindo novamente (80% treino, 20% teste)
train_new, test_new = train_test_split(novo_dataset_completo, test_size=0.2, random_state=15)

#redefinir indices
train_new = train_new.reset_index(drop=True)
test_new = test_new.reset_index(drop=True)
'''